In [1]:
import requests
import base64

CLIENT_ID = 'e11341ccd10344e18007deda153995fe'
CLIENT_SECRET = 'a4a4f56a54e84d55a3951c28f1bd9b1f'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [3]:
playlist_id = '0hlHhFARr6laW9oLbomqho'

# Load the music dataset
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                                           Track Name   
0                                              Demons  \
1                                             Thunder   
2                   INDUSTRY BABY (feat. Jack Harlow)   
3                                         Radioactive   
4                                             Hustler   
..                                                ...   
95                             Battle (feat. Faouzia)   
96                      Marry Me - Kat & Bastian Duet   
97  Symphony No. 5 in C Minor, Op. 67: I. Allegro ...   
98  Levitate - From The Original Motion Picture “P...   
99                          I Forgot That You Existed   

                                              Artists   
0                                     Imagine Dragons  \
1                                     Imagine Dragons   
2                              Lil Nas X, Jack Harlow   
3                     Imagine Dragons, Kendrick Lamar   
4                             

In [4]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [6]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span 
    weight = 1 / (time_span.days + 1)
    return weight

In [7]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [8]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [9]:
input_song_name = "Natural"
recommendations = content_based_recommendations(input_song_name, num_recommendations=5)
print(f"Recommended songs for '{input_song_name}':")
print(recommendations)

Recommended songs for 'Natural':
                                          Track Name   
92                         Havana (feat. Young Thug)  \
87  I Don’t Wanna Live Forever (Fifty Shades Darker)   
45                                     Looking at Me   
61                         Unholy (feat. Kim Petras)   
78                                       abc (nicer)   

                       Artists   
92  Camila Cabello, Young Thug  \
87          ZAYN, Taylor Swift   
45           Sabrina Carpenter   
61       Sam Smith, Kim Petras   
78                       GAYLE   

                                        Album Name Release Date  Popularity  
92                                          Camila   2018-01-12          80  
87  reputation Stadium Tour Surprise Song Playlist   2017-11-09          82  
45                                 Singular Act II   2019-07-19          74  
61                       Unholy (feat. Kim Petras)   2022-09-22          85  
78                                 

In [10]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    import pandas as pd

# Assuming 'hybrid_recommendations' is a DataFrame
    new_row = {
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }

    hybrid_recommendations = pd.concat([hybrid_recommendations, pd.DataFrame([new_row])], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations    

In [14]:
input_song_name = "Natural"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Recommended songs for '{input_song_name}':")
print(recommendations)

Recommended songs for 'Natural':
                                         Track Name   
3                         Unholy (feat. Kim Petras)  \
1  I Don’t Wanna Live Forever (Fifty Shades Darker)   
0                         Havana (feat. Young Thug)   
2                                     Looking at Me   
4                                       abc (nicer)   

                      Artists                                      Album Name   
3       Sam Smith, Kim Petras                       Unholy (feat. Kim Petras)  \
1          ZAYN, Taylor Swift  reputation Stadium Tour Surprise Song Playlist   
0  Camila Cabello, Young Thug                                          Camila   
2           Sabrina Carpenter                                 Singular Act II   
4                       GAYLE                                     abc (nicer)   

  Release Date  Popularity  
3   2022-09-22        85.0  
1   2017-11-09        82.0  
0   2018-01-12        80.0  
2   2019-07-19        74.0  
4   

In [25]:
import pandas as pd
from collections import Counter
from itertools import combinations

def apriori_recommendations(input_song_name, num_recommendations=5, min_support=0.01):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Create a basket of songs for each user (assuming one user for simplicity)
    basket = music_df['Track Name'].apply(lambda x: [x]).tolist()

    # Count song occurrences
    song_counts = Counter(song for transaction in basket for song in transaction)

    # Calculate the total number of transactions
    total_transactions = len(basket)

    # Find frequent songs based on min_support
    frequent_songs = [song for song, count in song_counts.items() if count / total_transactions >= min_support]

    # Remove the input song from frequent songs
    frequent_songs = [song for song in frequent_songs if song != input_song_name]

    # Find pairs of songs that occur together frequently
    frequent_pairs = []
    for transaction in basket:
        song_combinations = combinations(transaction, 2)
        frequent_pairs.extend(song_combinations)

    # Count pair occurrences
    pair_counts = Counter(frequent_pair for frequent_pair in frequent_pairs)

    # Find pairs with support greater than or equal to min_support
    recommended_pairs = [pair for pair, count in pair_counts.items() if count / total_transactions >= min_support]

    # Filter pairs to include the input song
    recommended_pairs = [pair for pair in recommended_pairs if input_song_name in pair]

    # Sort recommended pairs by support count
    recommended_pairs.sort(key=lambda pair: pair_counts[pair], reverse=True)

    # Get the top recommended pairs
    top_recommendations = recommended_pairs[:num_recommendations]

    # Extract the songs from the pairs and remove duplicates
    recommendations = list(set([song for pair in top_recommendations for song in pair if song != input_song_name]))

    return recommendations

input_song_name = "Thunder"
recommendations = apriori_recommendations(input_song_name, num_recommendations=10, min_support=0.005)
print(f"Apriori recommended songs for '{input_song_name}':")
print(recommendations)


Apriori recommended songs for 'Thunder':
[]


In [20]:
music_df.head(10)

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Demons,Imagine Dragons,Night Visions (Deluxe),1zcNZa89FXNruaesVKiVeu,5VC29kHMkzcaorzPKUqJbl,59,2012-09-04,175200,False,https://open.spotify.com/track/5VC29kHMkzcaorz...,...,0.710,3,-3.015,1,0.0321,0.19000,0.000250,0.3290,0.428,89.938
1,Thunder,Imagine Dragons,Evolve,33pt9HBdGlAbRGBHQgsZsU,1zB4vmk8tFRmM9UULNzbLB,87,2017-06-23,187147,False,https://open.spotify.com/track/1zB4vmk8tFRmM9U...,...,0.822,0,-4.833,1,0.0438,0.00671,0.134000,0.1470,0.288,167.997
2,INDUSTRY BABY (feat. Jack Harlow),"Lil Nas X, Jack Harlow",INDUSTRY BABY (feat. Jack Harlow),622NFw5Yk0OReMJ2XWcXUh,27NovPIUIRrOZoCHxABJwK,80,2021-07-23,212000,True,https://open.spotify.com/track/27NovPIUIRrOZoC...,...,0.704,3,-7.409,0,0.0615,0.02030,0.000000,0.0501,0.894,149.995
3,Radioactive,"Imagine Dragons, Kendrick Lamar",Radioactive,2M0IZTKgkN3ZpYluF4lKAM,69yfbpvmkIaB10msnKT7Q5,67,2014-01-01,276040,True,https://open.spotify.com/track/69yfbpvmkIaB10m...,...,0.894,9,-3.900,1,0.2230,0.04070,0.000000,0.2820,0.278,139.858
4,Hustler,Zayde Wølf,Golden Age,31QZh3KIkezhjDfmqDNnqy,6TyqP40vy0fskupWaEc4u5,54,2016-11-04,194772,False,https://open.spotify.com/track/6TyqP40vy0fskup...,...,0.857,11,-4.287,1,0.0456,0.03930,0.000000,0.0919,0.889,81.999
5,Copines,Aya Nakamura,NAKAMURA,3jqQFIXUakuDXdhFVvI7Ko,1Bhm5HNO1cq8olDbBmokyL,72,2018-11-02,171720,False,https://open.spotify.com/track/1Bhm5HNO1cq8olD...,...,0.787,8,-3.941,0,0.0855,0.27000,0.000000,0.1050,0.904,180.027
6,"Hey Mama (feat. Nicki Minaj, Bebe Rexha & Afro...","David Guetta, AFROJACK, Bebe Rexha, Nicki Minaj",Listen,77UW17CZFyCaRLHdHeofZu,285HeuLxsngjFn4GGegGNm,74,2014-11-21,192560,False,https://open.spotify.com/track/285HeuLxsngjFn4...,...,0.730,9,-4.091,1,0.1510,0.24000,0.000000,0.3250,0.525,85.979
7,Remember the Name (feat. Styles of Beyond),"Fort Minor, Styles Of Beyond",The Rising Tied,5v4Vx9loqMQCS3J7OmP9pa,6ndmKwWqMozN2tcZqzCX4K,73,2005-11-21,230493,True,https://open.spotify.com/track/6ndmKwWqMozN2tc...,...,0.835,8,-4.162,1,0.0911,0.05830,0.000003,0.0795,0.880,84.858
8,Do You Like Bass?,"Yellow Claw, Juyen Sebulba",Do You Like Bass?,4y3pQCy8fVOzaj2kxL59eL,4SJrFfeVaEWXdB5qUa6h1G,45,2017-09-08,195208,False,https://open.spotify.com/track/4SJrFfeVaEWXdB5...,...,0.983,1,-1.767,1,0.2990,0.00196,0.004700,0.2310,0.146,150.061
9,Bones,Imagine Dragons,Bones,1Q9SnHWPNEjVM0LrBFvJ1q,0HqZX76SFLDz2aW8aiqi7G,82,2022-03-11,165265,False,https://open.spotify.com/track/0HqZX76SFLDz2aW...,...,0.742,5,-3.678,0,0.0460,0.02060,0.000000,0.0754,0.571,114.071


In [38]:
import pandas as pd
from collections import Counter
from itertools import combinations

# Load your music dataset (music_df) here

def apriori_recommendations(input_song_name, num_recommendations=10, min_support=0.01):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Create a basket of songs for each user (assuming one user for simplicity)
    user_basket = music_df['Track Name'].tolist()

    # Count song occurrences
    song_counts = Counter(song for transaction in user_basket for song in transaction)

    # Calculate the total number of transactions
    total_transactions = len(user_basket)

    # Find frequent songs based on min_support
    frequent_songs = [song for song, count in song_counts.items() if (count / total_transactions) >= min_support]

    # Remove the input song from frequent songs
    frequent_songs = [song for song in frequent_songs if song != input_song_name]

    # Create a dictionary to store song co-occurrences
    song_co_occurrences = {}

    # Find pairs of songs that occur together frequently
    for transaction in user_basket:
        if input_song_name in transaction:
            for song in transaction:
                if song != input_song_name and song in frequent_songs:
                    if song in song_co_occurrences:
                        song_co_occurrences[song] += 1
                    else:
                        song_co_occurrences[song] = 1

    # Sort songs by co-occurrence count
    recommended_songs = sorted(song_co_occurrences, key=lambda song: song_co_occurrences[song], reverse=True)

    # Get the top recommended songs
    top_recommendations = recommended_songs[:num_recommendations]

    return top_recommendations

input_song_name = "Radioactive"
recommendations = apriori_recommendations(input_song_name, num_recommendations=11, min_support=0.01)
print(f"Apriori recommended songs for '{input_song_name}':")
print(recommendations)


Apriori recommended songs for 'Radioactive':
['a', 'i', 'R', 'd', 'o', 'c', 't', 'v', 'e']
